In [2]:
# Nov. 11, 2021, Xiaole Zhang
# Download meteo data from ecmwf 
# Dec, 29, 2021, Xiaole Zhang
# fix the date bugs when combine the files
# Apr. 28, 2023, Xiaole Zhang
# Get meteo data for Beijing ARG
# May 07, 2023, Xiaole Zhang
# fix the misalignment of the data
import cdsapi
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
import os
from pathlib import Path
import calendar 

# covert number to formated string
def number2String(f):
    return '%02d' % f

c = cdsapi.Client() 

# [up, left, down, right]
regionName = 'Chongqing'
regionBound = {'Beijing': [50, 105, 30, 127],\
               'Xian': [44.25, 97.5, 24.25, 119.5], \
              'Wuhan': [40.5, 104.5, 20.5, 124.5], \
              'Chongqing':[39.8, 96.5 ,19.8, 116.5]}

# data for Wuhan
initialYear = 2019
initialMonth = 12
numberOfMonths = 4
initialDate = datetime(initialYear, initialMonth, 1, 0) 

#targetYear = 2021
#targetMonths = [1,2]
#monthsList = list(map(number2String, targetMonths))

targetDays = range(1,32)
daysList = list(map(number2String, targetDays))

# to download the meteo day by month to meet the volume requirement
for currentMonthNum in range(numberOfMonths):
    currentDay = initialDate+ relativedelta(months=currentMonthNum)#date(targetYear, currentMonth, 1) 
    initalDay = currentDay 
    
    year = '%4d' % currentDay.year
    month = '%02d' % currentDay.month
    day = '%02d' % currentDay.day
    print('Current Day:' + currentDay.isoformat())
    
    # download single level data
    fileName = regionName + '/'+'SingleLevel_'+currentDay.strftime('%Y%m%d')+'.grib'
    filePath = Path(regionName + '/')
    filePath.mkdir(parents=True, exist_ok=True)
    
    c.retrieve( 
        'reanalysis-era5-single-levels', 
        { 
            'product_type': 'reanalysis', 
            'format': 'grib', 
            'variable': [ 
                '2m_temperature', 'boundary_layer_height', 'eastward_turbulent_surface_stress', 
                'evaporation', 'high_cloud_cover', 'medium_cloud_cover', 
                'northward_turbulent_surface_stress', 'skin_temperature', 'surface_sensible_heat_flux', 
                'surface_solar_radiation_downwards', 'surface_pressure', 'convective_precipitation','large_scale_precipitation','volumetric_soil_water_layer_1', 
            ], 
            'year': year, 
            'month': month,
            'day': daysList, 
            'time': [ 
                '00:00', '01:00', '02:00', 
                '03:00', '04:00', '05:00', 
                '06:00', '07:00', '08:00', 
                '09:00', '10:00', '11:00',
                '12:00', '13:00', '14:00',
                '15:00', '16:00', '17:00',
                '18:00', '19:00', '20:00',
                '21:00', '22:00', '23:00',
            ], 
            'area': regionBound[regionName], 
        }, 
        fileName) 
    
    # split the data to daily files
    ruleFile =regionName+'/rules_file'
    os.system("grib_filter "+ruleFile+" "+fileName)
    
    ###############################################
    # download 3d data
    fileName3d =  regionName + '/'+'PressureLevel_'+currentDay.strftime('%Y%m%d')+'.grib'
    
    c.retrieve( 
        'reanalysis-era5-pressure-levels', 
        { 
        'product_type': 'reanalysis', 
        'variable': [ 
        'specific_cloud_ice_water_content', 'specific_cloud_liquid_water_content', 'specific_humidity', 
        'temperature', 'u_component_of_wind', 'v_component_of_wind', 
        ], 
        'pressure_level': [ 
        '500', '550', '600', 
        '650', '700', '750', 
        '775', '800', '825', 
        '850', '875', '900', 
        '925', '950', '975', 
        '1000', 
        ], 
        'year': year, 
        'month': month,
        'day': daysList, 
        'time': [ 
        '00:00', '01:00', '02:00', 
        '03:00', '04:00', '05:00', 
        '06:00', '07:00', '08:00', 
        '09:00', '10:00', '11:00',
        '12:00', '13:00', '14:00',
        '15:00', '16:00', '17:00',
        '18:00', '19:00', '20:00',
        '21:00', '22:00', '23:00', 
        ], 
        'area': regionBound[regionName], 
        'format': 'grib', 
        }, 
        fileName3d) 
    
    # split the data to daily files
    ruleFile =regionName+'/rules_file_pressure'
    os.system("grib_filter "+ruleFile+" "+fileName3d)
    
    ###########################
    # combine the daily 2d and 3d data files into single files and delete the splitted data
    # need to install the grib api tools
    # sudo apt-get install libgrib-api-tools
    if(currentMonthNum!=0):      
        currentDay = initalDay + timedelta(days=-1)
        fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
        currentDay = initalDay + timedelta(hours=-6)
        fileName2dFc = regionName + '/' + 'ecmf_fc_'+currentDay.strftime('%Y%m%d_%-H')+'.grib'
        # ">>" should be used instead of ">", otherwise the input file will be erased
        os.system("cat "+fileNameAll+" "+fileName2dFc+" >> "+fileNameAll)

    dayRange = calendar.monthrange(initalDay.year, initalDay.month)
    for day in range(0, dayRange[1]):
        currentDay = initalDay + timedelta(days=day) 
        print('Current Day:' + currentDay.isoformat())
        fileName3d = regionName + '/' + 'ecmf_'+currentDay.strftime('%Y%m%d')+'_pressureLevels.grib'
        fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
        if(currentMonthNum==0 or day!=0):
            os.system("cat "+fileNameAll+" > "+fileNameAll)
        
        for hour in [-6, 6, 18]:
            currentDay = initalDay + timedelta(days=day) + timedelta(hours = hour)
            fileName2dFc = regionName + '/' + 'ecmf_fc_'+currentDay.strftime('%Y%m%d_%-H')+'.grib'
            os.system("cat "+fileNameAll+" "+fileName2dFc+" >> "+fileNameAll)
            if(hour!=18):
               os.system("rm "+fileName2dFc)  
        
        for hour in range(0, 24):
            currentDay = initalDay + timedelta(days=day) + timedelta(hours = hour)
            fileName2dAn = regionName + '/' + 'ecmf_an_'+currentDay.strftime('%Y%m%d_%-H')+'.grib'
            os.system("cat "+fileNameAll+" "+fileName2dAn+" >> "+fileNameAll)
            os.system("rm "+fileName2dAn)
            
        os.system("cat "+fileNameAll+" "+fileName3d+" >> "+fileNameAll)
        os.system("rm "+fileName3d) 
        
    currentDay = initalDay + timedelta(days=day+1)
    fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
    os.system("cat "+fileNameAll+" > "+fileNameAll)
    currentDay = initalDay + timedelta(days=day+1) + timedelta(hours = -6)
    fileName2dFc = regionName + '/' + 'ecmf_fc_'+currentDay.strftime('%Y%m%d_%-H')+'.grib'
    os.system("cat "+fileNameAll+" "+fileName2dFc+" >> "+fileNameAll)
    os.system("rm "+fileName2dFc)
        
     
   #fileNameAll = regionName + '/'+'ECMWF-'+currentDay.strftime('%Y%m%d')+'.grb'
   #os.system("cat "+fileName+" "+fileName3d+" > "+fileNameAll)
   #os.system("rm "+fileName+" "+fileName3d)

2023-07-03 16:23:35,623 INFO Welcome to the CDS
2023-07-03 16:23:35,624 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels


Current Day:2019-12-01T00:00:00


2023-07-03 16:23:35,984 INFO Request is queued
2023-07-03 16:23:37,012 INFO Request is running
2023-07-03 16:31:54,500 INFO Request is completed
2023-07-03 16:31:54,501 INFO Downloading https://download-0015-clone.copernicus-climate.eu/cache-compute-0015/cache/data7/adaptor.mars.internal-1688394216.3962362-11342-9-126ddf32-97aa-422d-a152-ae49ef677bd6.grib to Chongqing/SingleLevel_20191201.grib (132.3M)
2023-07-03 16:31:56,942 INFO Download rate 54.2M/s 
2023-07-03 16:31:59,613 INFO Welcome to the CDS
2023-07-03 16:31:59,614 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-07-03 16:31:59,661 INFO Request is queued
2023-07-03 16:32:00,686 INFO Request is running
2023-07-03 17:12:21,214 INFO Request is completed
2023-07-03 17:12:21,215 INFO Downloading https://download-0020.copernicus-climate.eu/cache-compute-0020/cache/data9/adaptor.mars.internal-1688394720.178223-1894-13-8d253695-0d5d-488b-a981-267764dd8f01.grib to Chongqing

Current Day:2019-12-01T00:00:00
Current Day:2019-12-02T00:00:00
Current Day:2019-12-03T00:00:00
Current Day:2019-12-04T00:00:00
Current Day:2019-12-05T00:00:00
Current Day:2019-12-06T00:00:00
Current Day:2019-12-07T00:00:00
Current Day:2019-12-08T00:00:00
Current Day:2019-12-09T00:00:00
Current Day:2019-12-10T00:00:00
Current Day:2019-12-11T00:00:00
Current Day:2019-12-12T00:00:00
Current Day:2019-12-13T00:00:00
Current Day:2019-12-14T00:00:00
Current Day:2019-12-15T00:00:00
Current Day:2019-12-16T00:00:00
Current Day:2019-12-17T00:00:00
Current Day:2019-12-18T00:00:00
Current Day:2019-12-19T00:00:00
Current Day:2019-12-20T00:00:00
Current Day:2019-12-21T00:00:00
Current Day:2019-12-22T00:00:00
Current Day:2019-12-23T00:00:00
Current Day:2019-12-24T00:00:00
Current Day:2019-12-25T00:00:00
Current Day:2019-12-26T00:00:00
Current Day:2019-12-27T00:00:00
Current Day:2019-12-28T00:00:00
Current Day:2019-12-29T00:00:00
Current Day:2019-12-30T00:00:00
Current Day:2019-12-31T00:00:00


2023-07-03 17:12:58,154 INFO Welcome to the CDS
2023-07-03 17:12:58,155 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-07-03 17:12:58,187 INFO Request is queued


...

Current Day:2020-02-01T00:00:00
Current Day:2020-02-02T00:00:00
Current Day:2020-02-03T00:00:00
Current Day:2020-02-04T00:00:00
Current Day:2020-02-05T00:00:00
Current Day:2020-02-06T00:00:00
Current Day:2020-02-07T00:00:00
Current Day:2020-02-08T00:00:00
Current Day:2020-02-09T00:00:00
Current Day:2020-02-10T00:00:00
Current Day:2020-02-11T00:00:00
Current Day:2020-02-12T00:00:00
Current Day:2020-02-13T00:00:00
Current Day:2020-02-14T00:00:00
Current Day:2020-02-15T00:00:00
Current Day:2020-02-16T00:00:00
Current Day:2020-02-17T00:00:00
Current Day:2020-02-18T00:00:00
Current Day:2020-02-19T00:00:00
Current Day:2020-02-20T00:00:00
Current Day:2020-02-21T00:00:00
Current Day:2020-02-22T00:00:00
Current Day:2020-02-23T00:00:00
Current Day:2020-02-24T00:00:00
Current Day:2020-02-25T00:00:00
Current Day:2020-02-26T00:00:00
Current Day:2020-02-27T00:00:00
Current Day:2020-02-28T00:00:00
Current Day:2020-02-29T00:00:00


2023-07-03 18:58:19,467 INFO Welcome to the CDS
2023-07-03 18:58:19,471 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-07-03 18:58:19,608 INFO Request is queued


Current Day:2020-03-01T00:00:00


2023-07-03 18:58:20,634 INFO Request is running
2023-07-03 19:06:38,109 INFO Request is completed
2023-07-03 19:06:38,109 INFO Downloading https://download-0002-clone.copernicus-climate.eu/cache-compute-0002/cache/data2/adaptor.mars.internal-1688403499.8308957-12498-12-2c553d23-ce4c-4472-ba3c-e88575a1f4f9.grib to Chongqing/SingleLevel_20200301.grib (132.3M)
2023-07-03 19:06:40,659 INFO Download rate 51.9M/s 
2023-07-03 19:06:43,440 INFO Welcome to the CDS
2023-07-03 19:06:43,441 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-pressure-levels
2023-07-03 19:06:43,514 INFO Request is queued
2023-07-03 19:06:44,541 INFO Request is running
2023-07-03 19:45:04,946 INFO Request is completed
2023-07-03 19:45:04,946 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data0/adaptor.mars.internal-1688404003.7114463-30190-11-50002583-6b17-445d-a8c6-2dcde8613480.grib to Chongqing/PressureLevel_20200301.grib (831M)
20

Current Day:2020-03-01T00:00:00
Current Day:2020-03-02T00:00:00
Current Day:2020-03-03T00:00:00
Current Day:2020-03-04T00:00:00
Current Day:2020-03-05T00:00:00
Current Day:2020-03-06T00:00:00
Current Day:2020-03-07T00:00:00
Current Day:2020-03-08T00:00:00
Current Day:2020-03-09T00:00:00
Current Day:2020-03-10T00:00:00
Current Day:2020-03-11T00:00:00
Current Day:2020-03-12T00:00:00
Current Day:2020-03-13T00:00:00
Current Day:2020-03-14T00:00:00
Current Day:2020-03-15T00:00:00
Current Day:2020-03-16T00:00:00
Current Day:2020-03-17T00:00:00
Current Day:2020-03-18T00:00:00
Current Day:2020-03-19T00:00:00
Current Day:2020-03-20T00:00:00
Current Day:2020-03-21T00:00:00
Current Day:2020-03-22T00:00:00
Current Day:2020-03-23T00:00:00
Current Day:2020-03-24T00:00:00
Current Day:2020-03-25T00:00:00
Current Day:2020-03-26T00:00:00
Current Day:2020-03-27T00:00:00
Current Day:2020-03-28T00:00:00
Current Day:2020-03-29T00:00:00
Current Day:2020-03-30T00:00:00
Current Day:2020-03-31T00:00:00


In [7]:
currentDay.strftime('%Y%m%d_%-H')

'20170131_0'

In [23]:
for hour in [-6, 0, 6]:
    print(hour)

-6
0
6
